<a href="https://colab.research.google.com/github/roitraining/SparkforDataEngineers/blob/Development/Ch02_DataFrames_Lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import sys
sys.path.append('/home/student/ROI/Spark')
from initspark import *
sc, spark, conf = initspark()

###***LAB:*** Use the regions and territories RDDs from the previous lab and convert them into DataFrames with meaningful schemas.


In [0]:
regions = sc.textFile('hdfs://localhost:9000/regions')
regions = regions.map(lambda x : x.split(',')).map(lambda x : (int(x[0]), x[1]))
regionsdf = regions.toDF('RegionID:int, RegionName:string')
regionsdf.show()

In [0]:
territories = sc.textFile('hdfs://localhost:9000/territories')
territories = territories.map(lambda x : x.split(',')).map(lambda x : (int(x[0]), x[1], int(x[2])))
territoriesdf = territories.toDF('TerritoryID:int, TerritoryName:string, RegionID: int')
territoriesdf.show()

###***LAB:*** Using the df3 DataFrame, answer the following questions:

*   How many Platinum card purchases were there with a discount above $100?
*   Find the ten biggest discount amounts earned by women and show just the purchase amount, discount, and date.



In [0]:
print(df3.where("CardType = 'Platinum' and Discount > 100").count())
df3.where("Gender = 'F'").orderBy('Amount', ascending = False).select('Amount', 'Discount', 'Date').take(10)

###***LAB:*** Read the Products file from the JSON folder and categories from the CSVHeaders folder, then join them displaying just the product and category IDs and names, and sort by categoryID then productID. 

**Hint:** Drop the ambiguous column after the join.

In [0]:
products = spark.read.json('/home/student/ROI/SparkProgram/datasets/northwind/JSON/products')
#products.show()
products.printSchema()

categories = spark.read.csv('/home/student/ROI/SparkProgram/datasets/northwind/CSVHeaders/categories', header = True, inferSchema = True)
#categories.show()
categories.printSchema()

c = categories
p = products
c.join(p, c.CategoryID == p.categoryid).drop(p.categoryid).select('CategoryID', 'CategoryName', 'productid', 'productname').orderBy('categoryid', 'productid').show()
